In [1]:
library(phyloseq, verbose=FALSE)
library(corncob, verbose=FALSE)
library(magrittr, verbose=FALSE)
library(ggplot2, verbose=FALSE)

Warning message:
“package ‘magrittr’ was built under R version 4.1.3”


In [2]:
load("../02-diversity/master_phyloseq.RData")
ps.dat
# head(sample_data(ps.dat))

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 14111 taxa and 1960 samples ]
sample_data() Sample Data:       [ 1960 samples by 26 sample variables ]
tax_table()   Taxonomy Table:    [ 14111 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 14111 tips and 14109 internal nodes ]

In [ ]:
# head(tax_table(ps.dat))
ps.dat.glom <- ps.dat %>%
                tax_glom("V8")
ps.dat.glom
# differential abundance analysis 
set.seed(234)
da_analysis <- differentialTest(formula = ~ aliquot_type,
                               phi.formula = ~ aliquot_type,
                               formula_null = ~ 1,
                               phi.formula_null = ~ aliquot_type,
                               test = "Wald",
                               boot = FALSE,
                               data = ps.dat.glom,
                               fdr_cutoff = 0.05)

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 628 taxa and 1960 samples ]
sample_data() Sample Data:       [ 1960 samples by 26 sample variables ]
tax_table()   Taxonomy Table:    [ 628 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 628 tips and 627 internal nodes ]

In [ ]:
level <- c("V8")
x <- da_analysis

signif_taxa <- otu_to_taxonomy(x$significant_taxa, x$data, level = level)
signif_taxa <- gsub("_", " ", signif_taxa) # NOTE: ASV label no longer informative after glom
signif_taxa
var_per_mod <- length(x$restrictions_DA) + length(x$restrictions_DV)
total_var_count <- length(signif_taxa) * var_per_mod
# initialize empty dataframe
df <- as.data.frame(matrix(NA, nrow = total_var_count, ncol = 5))
colnames(df) <- c("x", "xmin", "xmax", "taxa", "variable")

qval <- stats::qnorm(.975)
restricts_mu <- attr(x$restrictions_DA, "index")
restricts_phi <- attr(x$restrictions_DV, "index")

# loop through models to populate dataframe
count <- 1
for (i in 1:length(x$significant_models)) {
  # Below from print_summary_bbdml, just to get coefficient names
  tmp <- x$significant_models[[i]]
  coefs.mu <- tmp$coefficients[1:tmp$np.mu,, drop = FALSE]
  rownames(coefs.mu) <- paste0(substring(rownames(coefs.mu), 4), " Differential Abundance")
  coefs.mu <- coefs.mu[restricts_mu,, drop = FALSE]

  coefs.phi <- tmp$coefficients[(tmp$np.mu + 1):nrow(tmp$coefficients),, drop = FALSE]
  rownames(coefs.phi) <- paste0(substring(rownames(coefs.phi), 5), " Differential Variability")
  coefs.phi <- coefs.phi[restricts_phi - tmp$np.mu,, drop = FALSE]

  coefs <- rbind(coefs.mu, coefs.phi)
  for (j in 1:var_per_mod) {
    df[count, 1:3] <- c(coefs[j, 1], coefs[j, 1] - qval * coefs[j, 2],
                      coefs[j, 1] + qval * coefs[j, 2])
    df[count, 4:5] <- c(signif_taxa[i], rownames(coefs)[j])
    count <- count + 1
    }
}

head(df)
ggplot(df, aes(x = x, y = taxa)) +
  geom_vline(xintercept = 0, color = "gray50", lty = "dashed",
                          alpha = 0.75, lwd = 1) +
  geom_point() +
  geom_errorbarh(aes(xmin = xmin, xmax = xmax, colour = xmin<=0 & xmax <= 0| xmin>=0 & xmax >= 0), height = .3) +
  theme_bw() +
  facet_wrap(~variable, scales = "free_x", nrow = 1) +
  labs(title = "", x = "", y = "Taxa") +
  scale_y_discrete(limits = rev(df$taxa)) +
  scale_x_continuous(breaks = scales::pretty_breaks(n = 5)) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1), legend.position="none") # turned off legend

In [ ]:
# just H-CF versus D-CD
ps.dat.sub <- subset_samples(ps.dat.glom, aliquot_type == "D-CD" | aliquot_type == "H-CF")
da_analysis <- differentialTest(formula = ~ aliquot_type,
                               phi.formula = ~ aliquot_type,
                               formula_null = ~ 1,
                               phi.formula_null = ~ aliquot_type,
                               test = "Wald",
                               boot = FALSE,
                               data = ps.dat.sub,
                               fdr_cutoff = 0.05)

In [ ]:
# plot
level <- c("V8")
x <- da_analysis

signif_taxa <- otu_to_taxonomy(x$significant_taxa, x$data, level = level)
signif_taxa <- gsub("_", " ", signif_taxa) # NOTE: ASV label no longer informative after glom
signif_taxa
var_per_mod <- length(x$restrictions_DA) + length(x$restrictions_DV)
total_var_count <- length(signif_taxa) * var_per_mod
# initialize empty dataframe
df <- as.data.frame(matrix(NA, nrow = total_var_count, ncol = 5))
colnames(df) <- c("x", "xmin", "xmax", "taxa", "variable")

qval <- stats::qnorm(.975)
restricts_mu <- attr(x$restrictions_DA, "index")
restricts_phi <- attr(x$restrictions_DV, "index")

# loop through models to populate dataframe
count <- 1
for (i in 1:length(x$significant_models)) {
  # Below from print_summary_bbdml, just to get coefficient names
  tmp <- x$significant_models[[i]]
  coefs.mu <- tmp$coefficients[1:tmp$np.mu,, drop = FALSE]
  rownames(coefs.mu) <- paste0(substring(rownames(coefs.mu), 4), " Differential Abundance")
  coefs.mu <- coefs.mu[restricts_mu,, drop = FALSE]

  coefs.phi <- tmp$coefficients[(tmp$np.mu + 1):nrow(tmp$coefficients),, drop = FALSE]
  rownames(coefs.phi) <- paste0(substring(rownames(coefs.phi), 5), " Differential Variability")
  coefs.phi <- coefs.phi[restricts_phi - tmp$np.mu,, drop = FALSE]

  coefs <- rbind(coefs.mu, coefs.phi)
  for (j in 1:var_per_mod) {
    df[count, 1:3] <- c(coefs[j, 1], coefs[j, 1] - qval * coefs[j, 2],
                      coefs[j, 1] + qval * coefs[j, 2])
    df[count, 4:5] <- c(signif_taxa[i], rownames(coefs)[j])
    count <- count + 1
    }
}

head(df)
ggplot(df, aes(x = x, y = taxa)) +
  geom_vline(xintercept = 0, color = "gray50", lty = "dashed",
                          alpha = 0.75, lwd = 1) +
  geom_point() +
  geom_errorbarh(aes(xmin = xmin, xmax = xmax, colour = xmin<=0 & xmax <= 0| xmin>=0 & xmax >= 0), height = .3) +
  theme_bw() +
  facet_wrap(~variable, scales = "free_x", nrow = 1) +
  labs(title = "", x = "", y = "Taxa") +
  scale_y_discrete(limits = rev(df$taxa)) +
  scale_x_continuous(breaks = scales::pretty_breaks(n = 5)) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1), legend.position="none") # turned off legend

In [ ]:
pdf("img/corncob_hcf_v_dcd.pdf")
ggplot(df, aes(x = x, y = taxa)) +
  geom_vline(xintercept = 0, color = "gray50", lty = "dashed",
                          alpha = 0.75, lwd = 1) +
  geom_point() +
  geom_errorbarh(aes(xmin = xmin, xmax = xmax, colour = xmin<=0 & xmax <= 0| xmin>=0 & xmax >= 0), height = .3) +
  theme_bw() +
  facet_wrap(~variable, scales = "free_x", nrow = 1) +
  labs(title = "", x = "", y = "Taxa") +
  scale_y_discrete(limits = rev(df$taxa)) +
  scale_x_continuous(breaks = scales::pretty_breaks(n = 5)) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1), legend.position="none") # turned off legend
dev.off()

In [ ]:
# try doing coda balance analysis with the data at ASV level
# collapse data to roughly species level to minimize high sparsity
# temp <- tax_glom(ps.dat, taxrank=rank_names(ps.dat)[7])

In [ ]:
# save copy to reduce time on previous command
# glom <- temp
# remove any taxa with fewer than 10 counts and in at least 5% of samples post merging 
glom <- filter_taxa(ps.dat, function(x) sum(x > 10) > (0.05*length(x)), TRUE)
# only include taxa that are health or disease 
glom <- subset_samples(glom, aliquot_type == "D-CD" | aliquot_type == "H-CF")
# pull data
dat <- t(as.data.frame(otu_table(glom)))
map <- as.data.frame(as.matrix(sample_data(glom))) # have to coerce to data frame
map <- tibble::rownames_to_column(map) # retain rownames for downstream processing

In [ ]:
# get corresponding taxonomy name for each asv
taxa <- as(tax_table(glom), "matrix")
taxadf <- as.data.frame(taxa)
orderdf <- select(taxadf, V9)
orderdf <- orderdf %>%
    rownames_to_column(var = "ASV")

In [ ]:
# rename ASV at species level
dat <- as.data.frame(dat)
dat <- dat %>% 
    rownames_to_column(var = "ASV")
dat <- left_join(dat, orderdf, by=c('ASV'='ASV'))

In [ ]:
rownames(dat) <- paste(dat$V9, dat$ASV, sep="_")
dat <- dat[2:(length(dat)-1)] #remove last column
dat <- as.matrix(t(dat))
head(dat)

In [ ]:
library(dplyr)
# merge new metadata with asv table so the response variable is in the same order
datmerge <- merge(dat, map, by.x = "row.names", by.y = "rowname")
datmerge <- datmerge[!duplicated(datmerge[c('Row.names')]), ]
row.names(datmerge) <- datmerge$Row.names
# define data and response variable
dif <- dim(datmerge)[2] - dim(map)[2]
x <- datmerge[,2:dif]
# make sure only numeric data
x <- select_if(x, is.numeric)
dim(x)

In [ ]:
# define response variable 
y <- as.factor(datmerge$aliquot_type)

In [ ]:
# selbal takes forever and I keep running into errors, trying a different package
# install.packages("coda4microbiome")
library(coda4microbiome)
set.seed(852)

In [ ]:
bal <- coda_glmnet(x, y)


In [ ]:
bal$taxa.name

In [ ]:
pdf("balance_hcf_v_dcd.pdf", width = 10, height = 20)
bal$`predictions plot`
bal$`signature plot`
dev.off()